# Procesando archivos VTK desde Python

 Ejemplo para leer un archivo, aplicar filtros y guardar

In [1]:
# GUARDAR ARCHIVO en formato vtkPolydata DESPUÉS DE APLICAR FILTROS
import vtk
def leerArchivo(nombre_archivo):  
    reader = vtk.vtkPolyDataReader()             # SOURCE/READER                  
    reader.SetFileName(nombre_archivo)              
    reader.Update()                          
    data = reader.GetOutput()
    #FILTER
    bordes = vtk.vtkFeatureEdges()                     
    bordes.SetInputData(data)                    
    bordes.BoundaryEdgesOn()                           
    bordes.FeatureEdgesOn()
    bordes.ManifoldEdgesOn()
    bordes.NonManifoldEdgesOff()
    bordes.Update()           
    cc_filter = vtk.vtkPolyDataConnectivityFilter()    # creo un segundo fitro: connectivity
    cc_filter.SetInputData(bordes.GetOutput())
    cc_filter.ColorRegionsOn()
    cc_filter.SetExtractionModeToSpecifiedRegions()
    components = list()
    idx = 0
    while True:   #https://public.kitware.com/pipermail/vtkusers/2017-March/098063.html
        cc_filter.AddSpecifiedRegion(idx)
        cc_filter.Update()
        component = vtk.vtkPolyData()
        component.DeepCopy(cc_filter.GetOutput())  
        if component.GetNumberOfCells() <= 0:
            break     # corta cuando ya no hay celdas para comparar
        components.append(component)  #puedo usar 'return components' al final de la función
        cc_filter.DeleteSpecifiedRegion(idx)
        idx += 1 
    # write to file
    writer = vtk.vtkPolyDataWriter()
    writer.SetFileName('ejemplo.vtk')
    writer.SetInputData(cc_filter.GetOutput())
    writer.SetFileTypeToASCII()
    writer.Write()
comp=leerArchivo('tubes_n1_raw.vtk')

Acciones pendientes de resolver desde la librería:
* controlar la versión de salida del archivo escrito, por defecto **vtk DataFile Version 4.2**
* combinar la escritura con otro archivo anterior
*  asignar global_ids

Aparentemente, aunque la librería de VTK tiene funciones muy variadas, no es posible realizar todas las transformaciones deseadas desde ahí. Eso explicaría entre otras cosas la existencia de librerías separadas en Paraview e ITK, que se relacionan con VTK pero tienen implementacioens independientes

(Sigue pendiente explorar las funciones de filtros de Paraview desde python)
***

## Alternativas mixtas para procesar archivos vtk
A continucación alternativas que, si bien tratan de maximizar el uso de vtk, combinan con librerías exteriores.

Para esto veremos distintas funcions para interactuar con los formatos de datos de vtk

Primero lee el archivo, obtiene su salida, y se sacan algunos parámetros como la cantidad de nodos y elementos

In [2]:
import vtk
# SOURCE/READER 
reader = vtk.vtkPolyDataReader()                
reader.SetFileName('tubes_n1_raw.vtk')              
reader.Update()                          
data = reader.GetOutput()

N_points=data.GetNumberOfPoints()
N_cells=data.GetNumberOfCells()
N_points, N_cells

(26436, 52112)

La función GetPoint(int) toma el id de un nodo y devuelve su posición en [x,y,z]

In [3]:
#get point (x,y,z) by its id
data.GetPoint(3)

(-0.468221700423354, -0.147699486438929, -0.0301054537553065)

La función FindPoint(float, float, float)  toma una posición [x,y,z] y devuelve el id del nodo más cercano

In [4]:
#Find id of nearest point to a (x,y,z) point
data.FindPoint(3,3,2)

25879

Para saber si una posición en [x,y,z] es parte de los nodos de la malla: primero enncontramos el nodo más cercano y comparamos si es igual a la posición dada

In [5]:
P=(3,3,2)
data.GetPoint(data.FindPoint(P))==P

False

In [6]:
P=(-0.468221700423354, -0.147699486438929, -0.0301054537553065)
data.GetPoint(data.FindPoint(P))==P

True

Creando otro filtro para obtener los nodos de los bordes:

In [7]:
#FILTER
bordes = vtk.vtkFeatureEdges()                     
bordes.SetInputData(data)                    
bordes.BoundaryEdgesOn()                           
bordes.FeatureEdgesOn()
bordes.ManifoldEdgesOn()
bordes.NonManifoldEdgesOff()
bordes.Update()           
b=bordes.GetOutput()
N=b.GetNumberOfPoints()
N, b.FindPoint(0.5,0.5,0.5), b.GetPoint(656)

(768, 656, (0.106711137867821, 0.106711137867821, 0.5))

Ejemplo para obtener los ids globales de los nodos filtrados:

In [8]:
puntos_borde=bordes.GetOutput().GetPoints()
N=puntos_borde.GetNumberOfPoints()

import numpy as np
creases=np.zeros(N) #array with global ids of edges

for i in range (0,N):
    creases[i]=data.FindPoint(puntos_borde.GetPoint(i))
creases=np.sort(creases)  #ordeno los puntos

Esta implementación es muy rápida, ya que usa un solo loop en lenguaje de scripting y aprovecha las funciones GetPoint y FindPoint que son muy veloces
***

Creando Point Data:  **Creases ID**, **Constraint Tags** , **boundaries points**, **boundaries global ids**, **BDispl ids**


In [14]:
#creo todos los array del tamaño máximo N_points
creases_id=np.zeros(N_points, dtype=np.int)  #boolean array for each global node, 1=edge, 0!=edge
constraint_tags=np.zeros((N_points,3),dtype=np.int)  
boundaries_points=np.zeros((N_points,3))     #lista de puntos de los boundaries
boundaries_global_ids=np.zeros(N_points)
bDispl_ids=np.zeros(N_points)

bord=bordes.GetOutput()
b_counter=0
bDispl_counter=0

for i in range(0,N_points):                   # Para cada nodo de la malla
    p=data.GetPoint(i)                        # obtengo  su posición [x,y,z]
    val=bord.GetPoint(bord.FindPoint(p))==p   # verifico si ese nodo es del borde
    creases_id[i]= val                        # creases_id   =1 s está en el borde  =0 si no es borde
    top  =  p[2]==0.5                         # igual a 1  para el anillo del techo con z = 0.5 
    ceil =  p[2]==-0.5                        # igual a 1  para el anillo de la base con z = -0.5   
    if ceil:
        constraint_tags[i][:]=1,1,1           # se fijan constraint tags
        boundaries_points[b_counter]=p        # se agrega a boundaries_points
        boundaries_global_ids[b_counter]=i    # se guarda su id global
        b_counter+=1
    if top :
        constraint_tags[i][:]=1,1,1           # se fijan constraint tags
        boundaries_points[b_counter]=p        # se agrega a boundaries_points
        boundaries_global_ids[b_counter]=i    # se guarda su id global
        b_counter+=1
        bDispl_ids[bDispl_counter]=i          #los id de los boundaries top se agregan a bDispl_ids
        bDispl_counter+=1
        
#borro los elementos no usados de los array que fueron creados más grandes
boundaries_points=np.split(boundaries_points,[b_counter,N_points])[0]    
boudaries_global_ids=np.split(boundaries_global_ids,[b_counter,N_points])[0]
bDispl_ids=np.split(bDispl_ids,[bDispl_counter,N_points])[0]
len(creases_id)

26436

La implementación es eficaz, usa un solo loop en scripting, y crea todas las listas necesarias.
***

In [18]:
dataImporter = vtk.vtkImageImport()
data_string = creases_id.tostring()
dataImporter.CopyImportVoidPointer(data_string, len(data_string))
dataImporter.SetDataScalarTypeToUnsignedChar()
dataImporter.SetNumberOfScalarComponents(1)
dataImporter.SetDataExtent(0, 26436, 0,0,0,0)
dataImporter.SetWholeExtent(0, 26436,0,0,0,0)

arr=vtk.vtkDoubleArray()
arr.CopyImportVoidPointer(data_string, len(data_string))
arr.SetDataScalarTypeToUnsignedChar()
arr.SetNumberOfScalarComponents(1)
arr.SetNumberOfTuples(len(data_string))
arr.SetName("creases_ids")


defField->SetNumberOfTuples(nPts);
                defField->SetNumberOfComponents(DOF);
                defField->SetName("DisplacementField");
                defField->SetArray(u_nodes, nPts, DOF);

In [25]:
from numpy import *

d=np.arange(0,365)
th=360/365*d
R=1
x=R*cos(th)

y=R*sin(th)
P=[x,y]
P

[array([ 1.00000000e+00,  5.51778251e-01, -3.91081523e-01, -9.83358809e-01,
        -6.94110485e-01,  2.17368669e-01,  9.33989093e-01,  8.13341069e-01,
        -3.64212679e-02, -8.53533996e-01, -9.05501724e-01, -1.45738320e-01,
         7.44671253e-01,  9.67525124e-01,  3.23047388e-01, -6.11024077e-01,
        -9.97346982e-01, -4.89604671e-01,  4.57040564e-01,  9.93974758e-01,
         6.39866743e-01, -2.87845652e-01, -9.57520684e-01, -7.68832526e-01,
         1.09070551e-01,  8.89198042e-01,  8.72209730e-01,  7.33346776e-02,
        -7.91280770e-01, -9.46557716e-01, -2.53299154e-01,  6.67027788e-01,
         9.89402007e-01,  4.24833230e-01, -5.20574533e-01, -9.99316641e-01,
        -5.82227845e-01,  3.56795317e-01,  9.75971637e-01,  7.20244530e-01,
        -1.81141102e-01, -9.20143972e-01, -8.34289761e-01, -5.41919326e-04,
         8.33691722e-01,  9.20567841e-01,  1.82206905e-01, -7.19492226e-01,
        -9.76207230e-01, -3.57807610e-01,  5.81346315e-01,  9.99356116e-01,
         5.2

## Escribiendo los datos a un archivo VTK
A continuación se muestra la implementación usado Python para construir un archivo copatible con VTK.

Aunque sigue pendiente evaluar la escritura desde otras librerias como **Paraview**, esta implementación es eficaz ya que ser realizará una sola vez para crear el archivo de parámetros y permite tener un control total

In [27]:
with open("tubes_n1_raw.vtk") as f:
    with open("tubes_n1_param.vtk", "w") as f1:
        for line in f:                                                   #copiar el archivo Raw
            f1.write(line) 
        
        f1.write('POINT_DATA '+str(N_points)+'\n\n')                     # Encabezado POINT_DATA
        f1.write('FIELD FieldData 3\n\n')                                # Encabezado para ¿¿¿
        f1.write('CreasesId 1 '+str(len(creases_id))+' int')             # Encabezado CreasesID 
        for i in creases_id:                                             # Escribe cada linea de CresasesId
            f1.write(str(i)+'\n')
        f1.write('ConstraintTags 3 '+str(len(constraint_tags))+' int\n') # Encabezado ConstraintTags
        for i in constraint_tags:                                        # Escribe cada linea de contratin tags
            f1.write(np.array2string(i)[1:-1]+'\n')

Del mismo modo se pueden agregar Boundary points y Bdispl (pendiente)

## Para identificar non manifold nodes
Se crea un array que a cada nodo le asigna un Master:
* primero se buscan los nodos únicos
* si el nodo es único es su propio master
* si el nodo se repite:
 * el primer nodo es master
 * los siguientes son slaves, es decir tienen como master al primero en aparecer

In [1]:
import numpy as np
import vtk
from vtk.numpy_interface import dataset_adapter as dsa

reader = vtk.vtkPolyDataReader()             # SOURCE/READER 
reader.SetFileName('non_manifold.vtk')                   
reader.Update()                                          
data_vtk=reader.GetOutput()
data = dsa.WrapDataObject(data_vtk)          # convierte a array compatible con numpy

N_points=data.GetNumberOfPoints()            # número de nodos
Points=np.array(data.Points)                 # nodos

unq, index = np.unique(data.Points,          # filtro valores uniq del array de puntos
                       axis=0,               # compara filas enteras
                       return_index=True)    # índice de los no repetidos

master=np.zeros(N_points,dtype=int)
for i in range(0, N_points):                 # para cada nodo i
    if i in index:                           # si es único
        master[i]=i                          # su master es i
    else:                                    # si no es único
        for j in range(0,i):                 # comparo con c/ nodo anterior  j
            if np.all(Points[i]==Points[j]): # si es igual  al nodo j (el primero que aparece)
                master[i]=j                  # su master es j
                break                        # detengo el bucle for
master

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10,  0, 12, 13,  3, 15, 16,
        6, 18, 19,  0, 21, 22, 23, 24,  3, 26, 27, 28, 29,  6, 31, 32])

Comparo la lista de puntos con la lista de puntos referida a sus master, para verificar que sean iguales:

In [12]:
np.all(Points==Points[master])  # si da True el algoritmo para los masters funciona bien

True

El array obtenido con los masters de cada nodo es un numpy.array, entonces:
* lo convierto a objeto vtk
* se doy como argumento al PointData de la malla
* escribo el nuevo archivo

In [3]:
Master=dsa.numpyTovtkDataArray(master, name = "master" )   # convierto la lista master a un objeto vtkDataArray 
data_vtk.GetPointData().SetScalars(Master)                 # agrego en PointData al vtkArray llamado Master (NO ES EL CONVERTIDO A NUMPY)

writer = vtk.vtkPolyDataWriter()                           # creo el objeto PolyDataWriter
writer.SetFileName('non_manifold_masters.vtk')             # le pongo nombre al archivo de salida
writer.SetInputData(data_vtk)                              # le conecto los datos data_vtk
writer.SetFileType(1)                                      # opcional: set file type to ascii
writer.Write()                                             # escribo el archivo .vtk  Returns 1 on success and 0 on failure. 

1

Verificando el contenido del archivo guardado vemos los atributos Point Data agregados

In [11]:
!head non_manifold_masters.vtk && echo '\n...\n' && tail non_manifold_masters.vtk

# vtk DataFile Version 4.2
vtk output
ASCII
DATASET POLYDATA
POINTS 33 double
-2 -2 -2 0 -2 0 2 -2 2 
-2 0 -2 0 0 0 2 0 2 
-2 2 -2 0 2 0 2 2 2 
-6 -2 2 -4 -2 0 -2 -2 -2 
-6 0 2 -4 0 0 -2 0 -2 

...

3 30 25 26 
3 32 31 27 

POINT_DATA 33
SCALARS master vtktypeint64
LOOKUP_TABLE default
0 1 2 3 4 5 6 7 8 
9 10 0 12 13 3 15 16 6 
18 19 0 21 22 23 24 3 26 
27 28 29 6 31 32 
